<a href="https://colab.research.google.com/github/zyuzyunda/helper/blob/main/career_helper_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import requests
import pandas as pd
from google.colab import drive
import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def fetch_data(url, params=None):
    """Функция для безопасного запроса данных с hh.ru"""
    retries = 5  # Количество попыток
    for i in range(retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса: {e}, попытка {i+1} из {retries}")
            time.sleep(1)  # Подождать перед повторной попыткой
    return None

In [ ]:
def get_vacancy_ids(job_title):
    """Получение списка ID вакансий по ключевому слову"""
    vacancy_ids = []

    for page in range(PAGES_NUMBER):
        params = {
            "text": job_title,
            "area": AREA,
            "page": page,
            "per_page": 10,
            "search_field": "name"  # Ищем только в названии вакансии!
        }
        data = fetch_data(URL, params)

        if data and "items" in data:
            vacancy_ids.extend([item["id"] for item in data["items"]])

        time.sleep(1)  # Задержка между запросами, чтобы не забанили

    print(f"🔹 Найдено {len(vacancy_ids)} вакансий по запросу: {job_title}")
    return vacancy_ids

In [ ]:
def get_vacancy_data(vacancy_ids):
    """Получение детальной информации о вакансиях"""
    vacancies = []

    for vacancy_id in vacancy_ids:
        data = fetch_data(f"{URL}/{vacancy_id}")

        if not data:
            continue

        # --- Вставляем здесь извлечение отраслей ---
        specs = data.get("specializations", [])

        # --- Далее уже формируем словарь с заранее подготовленными переменными ---
        vacancies.append({
            "id": data.get("id"),
            "name": data.get("name"),
            "salary_from": data["salary"]["from"] if data.get("salary") else None,
            "salary_to": data["salary"]["to"] if data.get("salary") else None,
            "currency": data["salary"]["currency"] if data.get("salary") else None,
            "company": data["employer"]["name"] if data.get("employer") else None,
            "city": data["area"]["name"] if data.get("area") else None,
            "experience": data["experience"]["name"] if data.get("experience") else None,
            "employment": data["employment"]["name"] if data.get("employment") else None,
            "description": data.get("description"),
            "url": data.get("alternate_url"),
            "published_at": data.get("published_at", "")[:10],
            "key_skills": [skill["name"] for skill in data.get("key_skills", [])],
        })

        time.sleep(1)  # Задержка между запросами

    return vacancies

In [ ]:
def save_to_csv(vacancies, job_title):
    """Сохранение данных в CSV-файл"""
    folder_path = "/content/drive/My Drive/parser_data/"
    os.makedirs(folder_path, exist_ok=True)  # Создаст папку, если её нет

    today = datetime.datetime.today().strftime('%Y-%m-%d')
    file_name = f"HHRU_{job_title}_{today}.csv".replace(" ", "_")  # Подставляем ключевое слово
    full_path = os.path.join(folder_path, file_name)

    for vacancy in vacancies:
      vacancy["search_keyword"] = job_title

    df = pd.DataFrame(vacancies)  # Создаём DataFrame
    df.to_csv(full_path, index=False, encoding="utf-8")

    print(f"✅ Файл сохранён в: {full_path}")

In [ ]:
# Основной процесс
def main():
    print("🚀 Начинаем парсинг данных с hh.ru")

    for job_title in JOB_TITLES:
        print(f"🔍 Ищем вакансии по запросу: {job_title}")

        vacancy_ids = get_vacancy_ids(job_title)
        if not vacancy_ids:
            print(f"❌ Вакансии не найдены по запросу: {job_title}")
            continue

        vacancies = get_vacancy_data(vacancy_ids)
        save_to_csv(vacancies, job_title)

    print("🎉 Парсинг завершен!")

In [ ]:
# Конфигурация
URL = "https://api.hh.ru/vacancies"
"""
JOB_TITLES = [
    # Продуктовая аналитика
    "Продуктовый аналитик",

    # Анализ данных / Data Science
    "Аналитик данных",
    "Data Analyst",
    "Data Scientist",
    "Инженер данных",
    "Machine Learning Engineer",

    # Бизнес-аналитика и системный анализ
    "Системный аналитик",
    "Бизнес-аналитик",
    "Аналитик 1С",

    # BI и визуализация
    "BI-аналитик",

    # Цифровая и веб-аналитика
    "Web-аналитик",

    # Маркетинг
    "Маркетинговый аналитик",
    "Digital Marketing Analyst",

    # Финансовая аналитика
    "Финансовый аналитик",
    "Investment Analyst",
    "Финансовый аналитик в области данных",

    # Руководящие роли
    "Руководитель отдела аналитики",
    "Head of Analytics",
    "Data Analytics Manager"
]
"""
JOB_TITLES = [
    # Продуктовая аналитика
    "Продуктовый аналитик",

    # Анализ данных / Data Science
    "Аналитик данных",
    "Data Analyst",
    "Data Scientist"]

PAGES_NUMBER = 10  # Количество страниц (максимум 20)
AREA = 113  # Россия

In [ ]:
if __name__ == "__main__":
    main()

🚀 Начинаем парсинг данных с hh.ru
🔍 Ищем вакансии по запросу: Продуктовый аналитик
🔹 Найдено 100 вакансий по запросу: Продуктовый аналитик
✅ Файл сохранён в: /content/drive/My Drive/parser_data/HHRU_Продуктовый_аналитик_2025-04-22.csv
🔍 Ищем вакансии по запросу: Аналитик данных
🔹 Найдено 100 вакансий по запросу: Аналитик данных
✅ Файл сохранён в: /content/drive/My Drive/parser_data/HHRU_Аналитик_данных_2025-04-22.csv
🔍 Ищем вакансии по запросу: Data Analyst
🔹 Найдено 100 вакансий по запросу: Data Analyst
✅ Файл сохранён в: /content/drive/My Drive/parser_data/HHRU_Data_Analyst_2025-04-22.csv
🔍 Ищем вакансии по запросу: Data Scientist
🔹 Найдено 100 вакансий по запросу: Data Scientist
✅ Файл сохранён в: /content/drive/My Drive/parser_data/HHRU_Data_Scientist_2025-04-22.csv
🎉 Парсинг завершен!


In [ ]:
# Путь к папке в Google Диске
folder_path = "/content/drive/My Drive/parser_data/"

# Получаем список всех CSV-файлов
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Загружаем все CSV-файлы в список
dfs = []
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Объединяем в один DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Удаляем дубликаты по всем колонкам, кроме search_keyword
if 'search_keyword' in combined_df.columns:
    # Группируем по id и объединяем search_keyword через запятую
    final_df = (
        combined_df
        .groupby("id", as_index=False)
        .agg({
            **{col: 'first' for col in combined_df.columns if col not in ['id', 'search_keyword']},
            'search_keyword': lambda x: ', '.join(sorted(set(map(str, x))))
        })
    )
else:
    final_df = combined_df.drop_duplicates(subset="id")

# Выводим результат
print(f"✅ Загружено {len(files)} файлов, итоговая таблица: {final_df.shape}")


✅ Загружено 21 файлов, итоговая таблица: (2133, 16)


In [ ]:
final_df.head(5)

,id,name,salary_from,salary_to,currency,company,city,experience,employment,description,url,published_at,key_skills,search_keyword
0,73086362,Системный аналитик,NaN,75000.0,RUR,Команда Ф5,Чебоксары,Нет опыта,Полная занятость,<p><strong>Команда Ф5 - это аккредитованная IT...,https://hh.ru/vacancy/73086362,2025-04-10,"['Активные продажи', 'B2B Продажи', 'Навыки пр...",Системный аналитик
1,79382684,Консультант-аналитик 1С,70000.0,NaN,RUR,РИЦ-1С,Екатеринбург,От 1 года до 3 лет,Полная занятость,<p>Компания «РИЦ-1С» на протяжении 19 лет помо...,https://hh.ru/vacancy/79382684,2025-04-09,"['1С: Зарплата и кадры', '1С: Зарплата и управ...",Аналитик 1С
2,80569403,Системный аналитик (Госуслуги МО),NaN,NaN,None,Центр Методического Обеспечения Оптимизации Пр...,Москва,От 3 до 6 лет,Полная занятость,<p><strong>ИТ-компания</strong><strong> «МАКСИ...,https://hh.ru/vacancy/80569403,2025-04-10,"['XML', 'UML', 'BPMN', 'REST', 'SOAP', 'SQL', ...",Системный аналитик
3,90648349,Системный аналитик,250000.0,NaN,RUR,МТС,Москва,От 3 до 6 лет,Полная занятость,<p>Компания <strong>Гольфстрим</strong> более ...,https://hh.ru/vacancy/90648349,2025-04-01,"['Atlassian Jira', 'Системный анализ', 'UML', ...",Системный аналитик
4,92114457,Инженер по информационным базам данных и сопро...,70000.0,NaN,RUR,КрасАвиаПорт,Красноярск,От 1 года до 3 лет,Полная занятость,<strong>Обязанности:</strong> <ul> <li>участие...,https://hh.ru/vacancy/92114457,2025-04-18,[],Инженер данных


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2096 entries, 0 to 2095
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2096 non-null   int64  
 1   name            2096 non-null   object 
 2   salary_from     542 non-null    float64
 3   salary_to       373 non-null    float64
 4   currency        644 non-null    object 
 5   company         2096 non-null   object 
 6   city            2096 non-null   object 
 7   experience      2096 non-null   object 
 8   employment      2096 non-null   object 
 9   description     2096 non-null   object 
 10  url             2096 non-null   object 
 11  published_at    2096 non-null   object 
 12  key_skills      2096 non-null   object 
 13  search_keyword  2096 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 229.4+ KB


In [ ]:
# Указываем путь для итогового файла
output_path = os.path.join(folder_path, "HHRU_Объединённый_датасет_analyst4.csv")

# Сохраняем DataFrame в CSV
final_df.to_csv(output_path, index=False, encoding="utf-8")

print(f"📁 Итоговый файл сохранён: {output_path}")

📁 Итоговый файл сохранён: /content/drive/My Drive/parser_data/HHRU_Объединённый_датасет_analyst4.csv


In [ ]:
print(df.keys())
print("specializations:", df.get("specializations"))
print("specialization:",  df.get("specialization"))


Index(['id', 'name', 'salary_from', 'salary_to', 'currency', 'company', 'city',
       'experience', 'employment', 'description', 'url', 'published_at',
       'key_skills', 'industry', 'industry_list', 'search_keyword'],
      dtype='object')
specializations: None
specialization: None
